## load libraries and written functions

Note: If you do not install the RDKit python package, please go to the verano.py file and manually comment out any codes/imports relating to RDKit to avoid import errors in the line below. 

In [ ]:
from verano import *
%matplotlib inline

# Load dataset and descriptor names

In [ ]:
datasets=['Delaney','Test']

md_suffix=['67','PCAS','UD','PFAS','PC']

md_names=['RF & 67 Descriptors',
          'RF & PCA selected',
          'RF & Uncorrelated',
          'RF & PFA selected',
          'RF & Principal components',
          'Single decision tree']

## Load descriptor names


In [ ]:
file='./data/descriptors.txt'

with open(file,'r') as lines:
    descriptors=lines.readlines()

descriptors=[x.strip() for x in descriptors] 

print('The first five descriptors are:',descriptors[:5])

# Create standard descriptors

## Import datasets, create standard descriptors

Run this part to create the standard descriptors using the RDKit software library or go to section 1.3 to load the saved data directly

In [ ]:
cols=['name','logS','smiles']
data_up=[]

count=0
for i,dataset in enumerate(datasets):
    print('importing dataset',dataset)
    vars()[dataset]=pd.read_csv('./data/{0}.csv'.format(dataset))
    vars()[dataset].columns=cols
    # Convert all smiles strings to canonical format
    vars()[dataset]['smiles']=canonicals(vars()[dataset].smiles)
    vars()[dataset]['data']=[i]*len(vars()[dataset])
    data_up.append(vars()[dataset])
    count+=len(vars()[dataset])

print('Number of total compounds across all sets:',count)

# create one dataframe with all solubility data
mol_data=pd.concat(data_up,ignore_index=True)

In [ ]:
# Convert to rdkit format and add hydrogens
mol_rdkit=SMILES2MOLES(mol_data.smiles)
mol_h=ADDH_MOLES(mol_rdkit)

# Get 67 standard descriptors
mol_SD,del_idx=STDD(mol_h)
mol_descriptors=pd.DataFrame(data=mol_SD)

# Delete molecules whose descriptors could not be obtained
mol_data=mol_data.drop(del_idx)
mol_data=mol_data.reset_index(drop=True)

In [ ]:
# Remove any molecules having a nan as a value
mol_data,mol_descriptors=remove_nans(mol_data,mol_descriptors)

## Save data

Comment in this code to save the descriptors that were created to avoid having to reproduce them again. 

In [ ]:
# Save descriptors to save time 
# pkl.dump(mol_data,open('./data/pa_data.pkl','wb'))
# pkl.dump(mol_descriptors,open('./data/pa_descriptors.pkl','wb'))

## Load saved data 

In [ ]:
# mol_data=pkl.load(open('./data/pa_data.pkl','rb'))
# mol_descriptors=pkl.load(open('./data/pa_descriptors.pkl','rb'))

## Create lists to store the indices of the training/test data

In [ ]:
test_idx=list(mol_data.index[mol_data['data']==1])
train_idx=list(mol_data.index[mol_data['data']==0])

## Distribution of train/test set values

In [ ]:
test_mean=mol_data.loc[test_idx].logS.values.mean()
test_std=mol_data.loc[test_idx].logS.values.std()

In [ ]:
logs_values=[]
for dataset in ['train','test']:
    logs_values.append(mol_data.iloc[vars()[dataset+'_idx']].logS.values)
    print('{0}ing set has mean logS of {1}, std of {2}'.\
          format(dataset,
                 round(np.mean(logs_values[-1]),2),
                 round(np.std(logs_values[-1]),2)))

In [ ]:
fs=15
plt.figure(figsize=(6,4))
plt.ylim((-15,3))
plt.ylabel('log $S$',fontsize=fs)
plt.xlabel('Dataset',fontsize=fs)
ax=sns.boxplot(data=logs_values,showmeans=True,
              meanprops={"marker":"o",
                       "markerfacecolor":"white", 
                       "markeredgecolor":"black",
                      "markersize":"5"})
ax.set_xticklabels(datasets,fontsize=fs-4)
ax.set_yticklabels(ax.get_yticks(),size=fs-4)
plt.show()

# Principal component analysis

PCA is applied on the standardized training data. The model is then used to transform the standardized test data. 

In [ ]:
train_MD=mol_descriptors.iloc[train_idx]
test_MD=mol_descriptors.iloc[test_idx]

# Normalize or 'z-score' features
ss=StandardScaler()
train_scaled=ss.fit_transform(train_MD)
test_scaled=ss.transform(test_MD)

# PCA model
pca=PCA()
project_train=pca.fit_transform(train_scaled)
project_test=pca.transform(test_scaled)

## Proportion of variance explained 

Using the PCA model, we can see how many principal components (PCs) are required to have 90% of the cumulative proportion of variance explained (PVE) and the individual PVE values for those PC.

In [ ]:
N_pc=np.where(np.cumsum(pca.explained_variance_ratio_)>=0.9)[0][1]
print('90% variance explained by first {} PCs with each having PVE values of:'.format(N_pc))
print(pca.explained_variance_ratio_[:N_pc])

print('\nCumulative PVE of first {} PCs:'.format(N_pc))
print(np.cumsum(pca.explained_variance_ratio_)[:N_pc])

The PVE and cumulative PVE can be visualized using a scree plot which shows these values for the first 30 PCs. 

In [ ]:
fs=18
N_plot=30
fig,axes=plt.subplots(1,2,figsize=(12,5))

axes[0].plot(range(1,N_plot+1),pca.explained_variance_ratio_[:N_plot],'-o')
axes[0].set_ylabel('Proportion variance explained',fontsize=fs-2)
axes[0].set_xlabel('Principal Component',fontsize=fs-2)

axes[1].plot(range(1,N_plot+1),np.cumsum(pca.explained_variance_ratio_)[:N_plot],'-o')
axes[1].set_xlim(0,N_plot+1)
axes[1].set_ylim(0.36,1.01)
axes[1].set_ylabel('Cumulative prop variance explained',fontsize=fs-2)
axes[1].set_xlabel('Principal Component',fontsize=fs-2)

for i in range(2):
    axes[i].set_xticks(np.linspace(0,N_plot,16))
    for tick in axes[i].xaxis.get_major_ticks():
        tick.label.set_fontsize(fs*0.65) 
    for tick in axes[i].yaxis.get_major_ticks():
        tick.label.set_fontsize(fs*0.65) 
        
plt.tight_layout(rect=[0,0.03,1,0.95])
plt.show()

## Distribution of train/test datasets based on two principal components

We can compare how the training and test datasets are distributed using the information summarized within a pair of PCs and see how solubility relates to both via side-by-side plots. 

In [ ]:
# Change the values of pc1 and pc2 to visualize other PCs

pc1=0
pc2=1

fs=18
mapcolor='ocean_r' 
vmin=-11.5
vmax=1.5

projections=[project_train,project_test]
fig,axes=plt.subplots(1,2,figsize=(16*0.75,7*0.75))
for i,dataset in enumerate(['train','test']):
    ax=axes[i]
    projection=projections[i]
    x=projection[:,pc1]
    y=projection[:,pc2]
    c=mol_data.logS[vars()[dataset+'_idx']]
    vars()[dataset+'plot']=ax.scatter(x,y,
                    edgecolor='none',
                    alpha=0.8,
                    c=c,
                    cmap=plt.cm.get_cmap(mapcolor),
                    s=55,vmin=vmin,vmax=vmax)
    ax.set_xlabel('Principal Component {}'.format(pc1+1),fontsize=fs-2)
    ax.set_ylabel('Principal Component {}'.format(pc2+1),fontsize=fs-2)
    cbar=plt.colorbar(vars()[dataset+'plot'],ax=ax,fraction=0.03,pad=0.055)
    cbar.ax.tick_params(labelsize=11)
    cbar.ax.set_xlabel('log $S$',fontsize=fs*0.8)
    ax.set_title('{}'.format(datasets[i]),fontsize=fs)
    ax.set_xlim(-15,40)
    ax.set_ylim(-15,15)

plt.tight_layout()
plt.show()

## Descriptor loadings for each component

The individual PCs can be analyzed to see which descriptors are weighted the most based on their absolute 'loadings' with larger absolute loadings indicating a higher influence of that descriptor for that PC. 

In [ ]:
# Create a dataframe of the absolute loadings 
loadings=pd.DataFrame()
loadings_mat=abs(pca.components_)

for j in range(pca.n_components_):
    
    sorted_feats=[]
    for i in loadings_mat[j,:].argsort()[::-1]:
        sorted_feats.append((descriptors[i],loadings_mat[j,:][i]))
        
    loadings['PC'+str(j+1)]=sorted_feats

In [ ]:
loadings.head(10)

To determine how each of the descriptors relate to one another, we can use the loading values from a pair of PCs to create feature vectors that are visualized in a biplot. 

In [ ]:
# Create a dataframe of the loadings (but without absolute value)
loadings=pd.DataFrame()
loadings_mat=pca.components_

for j in range(pca.n_components_):
    
    sorted_feats=[]
    for i in loadings_mat[j,:].argsort()[::-1]:
        sorted_feats.append((descriptors[i],loadings_mat[j,:][i]))
        
    loadings['PC'+str(j+1)]=sorted_feats

In [ ]:
# Get loadings for individual descriptors to create biplot
pc1_idx=[]
pc2_idx=[]

for i in range(67):
    pc1_idx.append(loadings['PC1'][i][0])
    pc2_idx.append(loadings['PC2'][i][0])

In [ ]:
# Create the vector of loadings for four descriptors in v1
# Normalize each vector in v2
v1=[]
v2=[]

# Can change these descriptor names to look at other ones
desctype=['MolWt','MolMR','MolLogP','TPSA']

for i in desctype:
    idx1=pc1_idx.index(i)
    idx2=pc2_idx.index(i)
    v1.append([loadings['PC1'][idx1][1],loadings['PC2'][idx2][1]])
    v2.append(v1[-1]/np.linalg.norm(v1[-1]))

In [ ]:
for i in range(len(desctype)):
    print('The feature and normalized feature vectors for {}\n'.
          format(desctype[i]),v1[i],v2[i])

The dot product between two normalized feature vectors is indicative of their relationship:

1. 0: descriptors are independent
2. -1: linearly negatively correlated
3. 1: linearly positively correlated

For MolMR and MolWt, we see that they are very positively correlated. 

In [ ]:
print(np.dot(v2[0],v2[1]))

These vectors can be analyzed on a biplot and we can also see how the first two PCs compare to different descriptors in side-by-side plots.

In [ ]:
# Plot of lipophilicity against exact molecular weight
plot_idx=train_idx
feature_idx1=6
feature_idx2=4
s1=1
s2=-1
pc1=0
pc2=1

fs=18
fig,axes=plt.subplots(1,2,figsize=(16*0.75,7*0.75))
ax=axes[0]
x=project_train[:,pc1]
y=project_train[:,pc2]
c=mol_data.logS[plot_idx]
del_plot=ax.scatter(x,y,
                edgecolor='none',
                alpha=0.8,
                c=c,
                cmap=plt.cm.get_cmap(mapcolor),
                s=55,vmin=vmin,vmax=vmax)
ax.set_xlabel('First Principal Component',fontsize=fs-2)
ax.set_ylabel('Second Principal Component',fontsize=fs-2)
cbar=plt.colorbar(del_plot,ax=ax,fraction=0.03,pad=0.05)
cbar.ax.tick_params(labelsize=11)
cbar.ax.set_xlabel('log $S$',fontsize=fs*0.8)
ax.set_xlim(-12.5,30.5)
ax.set_ylim(-15,15)


V=np.array(v2)
c=['green','red','gold','dodgerblue']
origin=np.array([[0,0,0],[0,0,0]])
for i in range(4):
    ax.quiver(*origin,V[i,0],V[i,1],color=c[i],scale=2.5,label=desctype[i])
ax.set_xlabel('First principal component',fontsize=fs-2)
ax.set_ylabel('Second principal component',fontsize=fs-2)
ax.legend(loc=4)

featplot=axes[1].scatter(s1*mol_descriptors.iloc[plot_idx][feature_idx1],
                s2*mol_descriptors.iloc[plot_idx][feature_idx2],
            edgecolor='none',
            alpha=0.8,
            c=mol_data.iloc[plot_idx]['logS'],
            s=55,
            cmap=plt.cm.get_cmap(mapcolor),vmin=vmin,vmax=vmax) #,6))
axes[1].set_ylabel(descriptors[feature_idx2],fontsize=fs-2)
axes[1].set_xlabel(descriptors[feature_idx1],fontsize=fs-2)
cbar=plt.colorbar(featplot,fraction=0.03,pad=0.05,ax=axes[1])
cbar.ax.tick_params(labelsize=0.65*fs)
cbar.ax.set_xlabel('log $S$',fontsize=fs*0.8)
axes[1].set_xlim(-50,800)
axes[1].set_ylim(-10,10)

plt.tight_layout()
plt.show()

# Correlation analysis on Delaney dataset

Looking at the Pearson correlation coefficient (PCC) between pairs of descriptors presents another way to do dimensionality reduction. Here the heatmap shows the PCC between pairs of descriptors.

In [ ]:
md_Delaney=mol_descriptors.iloc[train_idx]
corr_mat=md_Delaney.corr()
corr_mat.columns=descriptors

plt.figure(figsize=(9*1.5,7.5*1.5),facecolor='w',edgecolor='k')
ax=sns.heatmap(corr_mat,linewidth=0.25,cmap='RdBu',
               cbar_kws={'shrink': 0.5,'label':'Pearson correlation'},
               xticklabels=corr_mat.columns,yticklabels=corr_mat.columns,
              vmin=-1,vmax=1)
ax.figure.axes[-1].yaxis.label.set_size(12)
ax.xaxis.set_ticks_position('top')
ax.xaxis.set_label_position('top')
plt.xticks(rotation='vertical')
plt.show()

## Find features with correlations greater than 0.80

We can find features that have absolute PCC greater than 0.80. 

In [ ]:
threshold=0.8

corr_np=np.array(abs(corr_mat))
np.fill_diagonal(corr_np,0)

# Where in the correlation matrix are values above threshold
corr_80=corr_np>=threshold

# List of the sets of correlated features
merge_corr=[]

for i in range(67):
    corrs=[]
    column=corr_80[:,i]
    if np.max(column)==1:
        corrs.append((descriptors[i],i))
        feats=np.where(column==1)
        for f in feats[0]:
            corrs.append((descriptors[f],f,corr_np[i,f]))
        merge_corr.append(corrs)

In [ ]:
indices=set()

for sets in merge_corr:
    for i,s in enumerate(sets):
        if i==0:
            print('Correlations for feature: ',s)
        else:
            print(s)
        indices.add(s[1])
    print('\n')
    
indices=list(indices)

print('Number of features correlated with Pearson coeff >= {0}: {1}'.format(threshold,len(indices)))

As many of the descriptors are highly correlated with another, we can remove those descriptors that have the greatest average PCC with all the rest until there is no longer between-descriptor PCC values greater than 0.80.

In [ ]:
Delaney_UD=uncorr_descriptor(mol_descriptors,train_idx,test_idx)
corr_mat=Delaney_UD.corr()
descriptors_uncorr=[descriptors[int(i)] for i in Delaney_UD.keys()]
corr_mat.columns=descriptors_uncorr

plt.figure(figsize=(9*0.8,7.5*0.8),facecolor='w',edgecolor='k')
ax=sns.heatmap(corr_mat,linewidth=0.5,cmap='RdBu',
               xticklabels=corr_mat.columns,yticklabels=corr_mat.columns,
               cbar_kws={'label':'Pearson correlation','shrink':0.6},
               vmin=-1,vmax=1)
ax.figure.axes[-1].yaxis.label.set_size(12)
ax.xaxis.set_ticks_position('top')
ax.xaxis.set_label_position('top')
plt.xticks(rotation='vertical')
plt.show()

# Prediction models using random forests

## Cross validation

The PCA and correlation analysis can be utilized to do dimensionality reduction and feature selection. Four subsets and versions of the original descriptors are created from that analysis:

1. Principal components representing 90% of the cumulative PVE
2. A subset of descriptors chosen based on their loadings for PCs representing 80% of the cumulative PVE
3. Principal feature selection based on the number of PCs representing 90% of the cumulative PVE 
4. Correlation analysis from section 3.1 

5-Fold cross-validation will be used to determine which of those four along with the full 67 descriptors will be effective. 

In [ ]:
# Dictionary to store results
cv_results={}

# Defining splits for cross validation
kf=KFold(n_splits=5,random_state=44,shuffle=True)
X=mol_descriptors.iloc[train_idx]
Y=mol_data.iloc[train_idx]

for j,dtype in enumerate(md_suffix):
    print('Cross validation on {}'.format(md_names[j]))
    
    for i,[train_index,test_index] in enumerate(kf.split(X,Y)):

        train_index=list(train_index)
        test_index=list(test_index)
        
        X_desc=get_descriptor(dtype,X,train_index,test_index)
        Xtrain_kf=X_desc.loc[train_index]
        Xtest_kf=X_desc.loc[test_index]
        Ytrain_kf=mol_data.iloc[train_index]['logS'].values
        Ytest_kf=mol_data.iloc[test_index]['logS'].values
        scaler=StandardScaler()
        Ytrain_ss=scaler.fit_transform(np.array(Ytrain_kf).reshape(-1,1))
        
        rf=RandomForestRegressor(n_estimators=500,random_state=10)
        rf.fit(Xtrain_kf,np.ravel(Ytrain_ss))
        yhat_kf=scaler.inverse_transform(rf.predict(Xtest_kf))
        cv_results[dtype+str(i)]=[Ytest_kf,yhat_kf]
        print('Completed fold',i)

### Cross-validation results

Consolidate cross-validation the mean results of each fold into a dataframe. Results are evaluated based on two metrics:

1. RMSE
2. Percent accuracy based on number of predictions being within $\pm$ of 0.7 log$S$ units 

In [ ]:
cv_columns=['Descriptor','RMSE','RMSE/SD','logS_pm0.7']
cv_metrics=pd.DataFrame(columns=cv_columns)
     
for i,dtype in enumerate(md_suffix):
    kf_RMSE=[]
    kf_RMSE_std=[]
    kf_logS_pm7=[]

    r_idx=5*i+j
    row=[md_names[i]]

    for k in range(5):
        Ytest_kf=cv_results[dtype+str(k)][0]
        yhat_kf=cv_results[dtype+str(k)][1]
        diff=abs(Ytest_kf-yhat_kf)
        lt7=np.sum(diff<=0.7)/len(Ytest_kf)*100

        kf_RMSE.append(np.sqrt(mse(Ytest_kf,yhat_kf)))
        kf_RMSE_std.append(kf_RMSE[-1]/np.std(Ytest_kf))
        kf_logS_pm7.append(lt7)

    row.append(np.mean(kf_RMSE))
    row.append(np.mean(kf_RMSE_std))
    row.append(np.mean(kf_logS_pm7))
    cv_metrics.loc[r_idx]=row

In [ ]:
cv_metrics.sort_values('RMSE')

## Random forest models

Having determine which of the descriptors are the most suitable, we can choose the best ones for predicting on the test set. Since we're only dealing with one algorithm and one training dataset here, we can quickly run the RF model on all five.

In [ ]:
# dictionary to store results
rf_results={}

for i,dtype in enumerate(md_suffix):
    print('Fitting RF for {}'.format(md_names[i]))
    vars()['Delaney_'+dtype]=get_descriptor(dtype,mol_descriptors,train_idx,test_idx)
    Xtrain=vars()['Delaney_'+dtype].iloc[train_idx]
    Ytrain=mol_data['logS'].iloc[train_idx]
    Xtest=vars()['Delaney_'+dtype].iloc[test_idx]
    Ytest=mol_data['logS'].iloc[test_idx]
    rf_results['Delaney_'+dtype]=rf_model(Xtrain,Xtest,Ytrain,Ytest,trees=500)

For purposes of comparison, we train a single decision tree model to make predictions and see how much predictive accuracy is gained from using an ensemble-based method such as RFs.

In [ ]:
# Train a single tree using 67 descriptors for comparison
Xtrain=Delaney_67.iloc[train_idx]
Ytrain=mol_data['logS'].iloc[train_idx]
Xtest=Delaney_67.iloc[test_idx]
Ytest=mol_data['logS'].iloc[test_idx]
Delaney_tree=dtree_model(Xtrain,Xtest,Ytrain,Ytest)

### Model results

Consolidate RF results and the single decision tree performance into a dataframe.

In [ ]:
rf_columns=['Descriptor','Length','RMSE','RMSE/SD','logS_pm0.7']
rf_metrics=pd.DataFrame(columns=rf_columns)
yhats=[]

for i,dtype in enumerate(md_suffix):
    
    yhats.append(rf_results['Delaney_'+dtype][0])
    diff=abs(yhats[-1]-np.array(Ytest))
    row=[md_names[i],
         vars()['Delaney_'+dtype].shape[1],
         np.sqrt(mse(yhats[-1],Ytest))]
    row.append(row[-1]/test_std)
    row.append(np.sum(diff<=0.7)/len(Ytest)*100)
    rf_metrics.loc[i]=row
    
    
yhats.append(Delaney_tree[0])
diff=abs(yhats[-1]-np.array(Ytest))
row=[md_names[5],
     Delaney_67.shape[1],
     np.sqrt(mse(yhats[-1],Ytest))]
row.append(row[-1]/test_std)
row.append(np.sum(diff<=0.7)/len(Ytest)*100)
rf_metrics.loc[5]=row

In [ ]:
rf_metrics.sort_values('RMSE')

## Plot of model predictions

The results of each model are plotted to see how accurate each descriptor/model performed.

In [ ]:
fs=14
lb=-8.5
ub=0

axlabels=['a','b','c','d','e','f']
color='tab:blue'
ms=25

cols=3
rows=2

fig,axes=plt.subplots(nrows=rows,ncols=cols,figsize=(12,8.5))
for i in range(6):
    ax=axes[int(i/cols)][i%cols]
    ax.scatter(Ytest,yhats[i],
               s=ms,
               color=color,
               facecolors=color)     
    resultsplot_setup(ax,fs,lb,ub,md_names[i],axlabels[i],weight='bold')

fig.tight_layout()
plt.show()

# Hard to predict molecules

In [ ]:
errors=[]
for i in range(5):
    errors.append(abs(yhats[i]-Ytest))
    
errors=np.array(errors)
MAE=errors.mean(axis=0)

sort_idx=np.argsort(MAE)[::-1]
err_idx=np.array(test_idx)[sort_idx]

In [ ]:
data_test=mol_data.iloc[test_idx].copy()
data_test['MAE']=MAE
data_test.sort_values('MAE',ascending=False).head(15)

In [ ]:
plt.figure(figsize=(6,4))

for i in range(5):
    plt.scatter(Ytest,errors[i],s=30,color='tab:blue',marker='x',lw=1)

plt.ylim(-0.25,5)
plt.xlim(-7.25,-0.75)
plt.xlabel('log $S$',fontsize=fs-2)
plt.ylabel('$abs($error$)$',fontsize=fs-2)
plt.setp(ax.get_xticklabels(),fontsize=fs-4)
plt.setp(ax.get_yticklabels(),fontsize=fs-4)
plt.show()

# Uncertainty quantification

Since RFs utilize a collection of decision trees, the predictions for the individual trees can be utilized to calculate an uncertainty value regarding each prediction called the 'bootstrap variance'. The variance and standard deviation for each set of predictions are calculated. Based on the standard deviation and assuming normality, a 95% prediction interval can be calculated and visualized to see how reliable is each prediction.

In [ ]:
model=rf_results['Delaney_PCAS'][1]
yhat_trees=np.zeros((500,100))

for i,tree in enumerate(model.estimators_):
    yhat_trees[i,:]=tree.predict(Delaney_PCAS.iloc[test_idx])

yh_var=yhat_trees.std(axis=0)
yh_std=yhat_trees.std(axis=0)

In [ ]:
# Calculate intervals  
percent=95
left=[]
right=[]
interval=np.zeros((100,3))
interval[:,0]=rf_results['Delaney_UD'][0]
for i in range(len(Ytest)):
    interval[i,1]=interval[i,0]-yh_std[i]*1.96
    interval[i,2]=interval[i,0]+yh_std[i]*1.96
    
pred_interval=pd.DataFrame(data=interval,
                           columns=['logS','lower','upper'])

In [ ]:
Y_sort=np.sort(Ytest)
Y_sort_idx=np.array(np.argsort(Ytest))
X=range(100)

plt.figure(figsize=(12,8))
plt.plot(X,Y_sort,'o',color='gold',label='Experimental log $S$')
plt.plot(X,interval[:,0][Y_sort_idx],'o',color='tab:blue',
         label='Predicted log $S$',alpha=0.75)
for i in range(100):
    lbnd=interval[Y_sort_idx[i],1]
    ubnd=interval[Y_sort_idx[i],2]
    plt.plot(X[i]*np.ones(20),np.linspace(lbnd,ubnd,20),color='lightblue',lw=4,zorder=0)
plt.legend(loc=4)
plt.ylabel('log $S$',fontsize=14)
plt.xlabel('Ordered molecules',fontsize=14)
plt.savefig('uq.png',dpi=250)
plt.show()